In [2]:
from djitellopy import Tello
import cv2
import numpy as np
import time
import math

def detection_center(det):
    """Center x, y normalized to [-0.5, 0.5]"""
    cx = (det[3] + det[5]) / 2.0 - 0.5
    cy = (det[4] + det[6]) / 2.0 - 0.5
    return (cx, cy)

def norm(vec):
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    best_det = None
    min_dist = float('inf')
    for det in detections[0, 0]:
        if det[2] > 0.4 and int(det[1]) == 1:  # 'person'
            center = detection_center(det)
            distance = norm(center)
            if distance < min_dist:
                min_dist = distance
                best_det = det
    return best_det

def circle_around_person(center_x, center_y, radius, speed=20, num_steps=36):
    """
    Make the drone move in a circle around a point (center_x, center_y) with a given radius.
    
    :param center_x: x-coordinate of the center point (person)
    :param center_y: y-coordinate of the center point (person)
    :param radius: radius of the circle to fly around
    :param speed: speed of the drone's movement (cm per move)
    :param num_steps: number of steps to complete the circle (higher for smoother, slower)
    """
    # Create circular motion using trigonometry
    angle_step = 360 / num_steps  # degrees per step
    
    for step in range(num_steps):
        # Calculate the angle for this step (in radians)
        angle = math.radians(angle_step * step)
        
        # Calculate the new position in x and y based on circle equation
        move_x = int(center_x + radius * math.cos(angle))
        move_y = int(center_y + radius * math.sin(angle))
        
        # Move drone in the direction of the calculated position
        x_offset = move_x - w // 2  # Calculate horizontal offset from frame center

        if abs(x_offset) > pixel_threshold:
            if x_offset > 0:
                print(f"Moving right {speed} cm")
                tello.move_right(speed)
            else:
                print(f"Moving left {speed} cm")
                tello.move_left(speed)

        # Optional: add some delay to smooth out the motion
        time.sleep(0.1)

# Load COCO class names
with open('COCO/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')

COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

model = cv2.dnn.readNet(model='COCO/frozen_inference_graph.pb',
                        config='COCO/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt',
                        framework='TensorFlow')

tello = Tello()
print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()

print("Press 't' to takeoff, 'q' to land and quit.")
drone_in_air = False

# Control parameters
area_target = 0.25
area_tolerance = 0.05
move_step_cm = 30
pixel_threshold = 40  # pixels for left/right decision

last_move_time = time.time()
move_interval = 0.8  # seconds between moves

try:
    while True:
        frame = tello.get_frame_read().frame
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape

        # Object detection
        blob = cv2.dnn.blobFromImage(frame, size=(300, 300), mean=(104, 117, 123), swapRB=True)
        model.setInput(blob)
        detections = model.forward()
        det = closest_detection(detections)

        if det is not None:
            class_id = int(det[1])
            class_name = class_names[class_id - 1]
            color = COLORS[class_id]

            x1 = int(det[3] * w)
            y1 = int(det[4] * h)
            x2 = int(det[5] * w)
            y2 = int(det[6] * h)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            label = f"{class_name} {det[2]:.2f}"
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

            # Calculate the center of the bounding box
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            
            # Trigger the circular movement when 'c' is pressed
            key = cv2.waitKey(1) & 0xFF
            if key == ord('c'):
                print("Starting to circle around the person...")
                radius = 50  # You can change the radius to adjust how wide the circle is
                circle_around_person(center_x, center_y, radius)

        # Show video
        cv2.imshow("Tello Tracking Feed", frame)

        # Controls
        key = cv2.waitKey(1) & 0xFF

        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne.")
            tello.move_up(50)

        if key == ord('q'):
            if drone_in_air:
                tello.land()
                print("Drone has landed.")
            break

except Exception as e:
    print(f"Error: {e}")
    if drone_in_air:
        tello.land()

except KeyboardInterrupt:
    print("Interrupted. Landing drone.")
    if drone_in_air:
        tello.land()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()

[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'streamon'


Connecting to Tello...
Battery level: 38%


[INFO] tello.py - 462 - Response streamon: 'ok'


Press 't' to takeoff, 'q' to land and quit.


2025-04-19 19:34:37.396 Python[20269:4231974] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
[INFO] tello.py - 438 - Send command: 'takeoff'
[INFO] tello.py - 462 - Response takeoff: 'ok'
[INFO] tello.py - 438 - Send command: 'up 50'


Drone is now airborne.


[INFO] tello.py - 462 - Response up 50: 'ok'
[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'error'
[INFO] tello.py - 438 - Send command: 'land'


Interrupted. Landing drone.


[INFO] tello.py - 462 - Response land: 'error'
[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'error'
[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'ok'


TelloException: Command 'land' was unsuccessful for 4 tries. Latest response:	'error'